In [11]:
import serial.tools.list_ports
import threading
import time

In [6]:
# Get a list of available ports
available_ports = serial.tools.list_ports.comports()

# Print the available ports
for port in available_ports:
    print(port.device)

/dev/cu.BoseMicroAndres
/dev/cu.Bluetooth-Incoming-Port


In [17]:
port = '/dev/cu.Bluetooth-Incoming-Port'  # Replace with the appropriate port name
ser = serial.Serial(port=port, baudrate=9600)

while (True):
    # Check if incoming bytes are waiting to be read from the serial input 
    # buffer.
    # NB: for PySerial v3.0 or later, use property `in_waiting` instead of
    # function `inWaiting()` below!
    if (ser.inWaiting() > 0):
        print('serial found')
        # read the bytes and convert from binary array to ASCII
        data_str = ser.read(ser.inWaiting()).decode('ascii') 
        # print the incoming string without putting a new-line
        # ('\n') automatically after every print()
        print(data_str, end='') 
    else:
        print('no serial found')

    # Put the rest of your code you want here
    
    # Optional, but recommended: sleep 10 ms (0.01 sec) once per loop to let 
    # other threads on your PC run during this time. 
    time.sleep(1)

no serial found
no serial found
no serial found
no serial found
no serial found


KeyboardInterrupt: 

### Main program

Suppositions about communication with arm/camera system.
<br>
<br>
System inputs:
- `start`: arm system should start looking for cotton balls and navigation system is ready to receive commands
- `stop`: arm system should pause the search for cotton balls. Navigation system won’t be listening to any commands

System outputs:
- `backward`/`forward`: move backward/forward
- `right`/`left`: move right/left
- `stop`: stop current navigation
- `finish`: cotton ball collection finished, main navigation can continue

Camera is expected to always stop a navigation started by it. For example if the camera told the navigation system to move `forward`, navigation will continue indefinitely until a `stop` command is received. If this is a problem, we can make the navigation work with timeouts.
<br>
Another thing that shouldn’t happen is for the arm system to send a `finish` command after a navigation command. For example send `forward` and then `finish`. The `stop` command is expected to always come before the `finish` command.


In [23]:
def getDistanceFromWall(test_returnValue, wallDirection, thetaRange = 3):
    time.sleep(0.5)
    print(f"getting distance from wall for theta: {wallDirection}")
    return test_returnValue

def navigateUsingWall(wallDirection, targetDistance):
    distance = 20
    currentDistance = getDistanceFromWall(distance, wallDirection)
    
    # 1 implies move closer to wall, -1 implies move away from the wall
    navDirection = int(math.copysign(1,currentDistance - targetDistance))
    
    print(f"starting navigation in direction: {navDirection}")
    
    while abs(currentDistance - targetDistance) > 1:
        print(f"current ditance to wall: {currentDistance}")
        distance = distance - navDirection
        currentDistance = getDistanceFromWall(distance, wallDirection, thetaRange = 3)
    
    print("navigation ended")
    
def navigateUsingCamera():
    # ping camera
    cameraCommand = input("enter camera command: ")
    
    while cameraCommand != "finished":
        if cameraCommand == "0":
            print("navigating south")
        elif cameraCommand == "90":
            print("navigating east")
        elif cameraCommand == "180":
            print("navigating north")
        elif cameraCommand == "270":
            print("navigating west")
        elif cameraCommand == "exit":
            break
        
        cameraCommand = input("enter camera command: ")
            
    print("camera navigation finished")
    return
    
def setOrientation(wallDirection):
    print(f"setting orientation using direction: {wallDirection}")
    return

In [22]:
cycles = [
    {
        "name": "cycle 0",
        "endOfCycleDistance": 20,
        "finalPoint": [
            {
                # horizontal navigation
                "name": "start point horizontal navigation",
                "lidarDirection": 270,
                "targetDistance": 10
            },
            {
                # vertical navigation
                "name": "start point vertical navigation",
                "lidarDirection": 0,
                "targetDistance": 40
            }
        ]
    },
    {
        "name": "cycle 1",
        "endOfCycleDistance": 20,
        "finalPoint": [
            {
                # horizontal navigation
                "name": "start point horizontal navigation",
                "lidarDirection": 90,
                "targetDistance": 10
            },
            {
                # vertical navigation
                "name": "start point vertical navigation",
                "lidarDirection": 0,
                "targetDistance": 40
            },
        ],
    },
    {
        "name": "cycle 2",
        "endOfCycleDistance": 20,
        "finalPoint": [
            {
                # horizontal navigation
                "name": "start point horizontal navigation",
                "lidarDirection": 270,
                "targetDistance": 10
            },
            {
                # vertical navigation
                "name": "start point vertical navigation",
                "lidarDirection": 180,
                "targetDistance": 40
            },
        ],
    },
    {
        "name": "cycle 3",
        "endOfCycleDistance": 20,
        "finalPoint": [
            {
                # horizontal navigation
                "name": "start point horizontal navigation",
                "lidarDirection": 90,
                "targetDistance": 10
            },
            {
                # vertical navigation
                "name": "start point vertical navigation",
                "lidarDirection": 180,
                "targetDistance": 40
            },
        ],
    },
    {
        "name": "cycle 4",
        "endOfCycleDistance": 20,
        "finalPoint": [
            {
                # horizontal navigation
                "name": "start point horizontal navigation",
                "lidarDirection": 270,
                "targetDistance": 10
            },
            {
                # vertical navigation
                "name": "start point vertical navigation",
                "lidarDirection": 180,
                "targetDistance": 10
            },
        ],
    },
    {
        "name": "cycle 5",
        "endOfCycleDistance": 20,
        "finalPoint": [
            {
                # horizontal navigation
                "name": "start point horizontal navigation",
                "lidarDirection": 90,
                "targetDistance": 10
            },
            {
                # vertical navigation
                "name": "start point vertical navigation",
                "lidarDirection": 180,
                "targetDistance": 10
            },
        ],
    },
    
]

In [37]:
def setOrientationUsingWall(direction):
    print(f"Setting orientation using {direction} direction")
    return

def navigationCommand(command, magnitude, timeOut=-1):
    if timeOut>0:
        print(f"/nStarting timed out navigation: ({command}, {timeOut})")
        time.sleep(timeOut)
    else:
        print(f"/nStarting indefinitely navigation: {command}")
    return

def sendCameraCommand(command):
    print("/nSending command to camera: {command}")
    
def getLidarData():
    '''
    This method gets lidar data (polar coordinates) and returns it as a numpy array with dimensions (N,2). The first entry
    of each element corresponds to the r and the second to theta
    '''
    return np.array([[0,0]])
    
def isEndOfCycle(lidarData, direction, threshold, mockValue=True):
    '''
    This method process lidar data and returns True when the distance of the lidar and the wall (in the specified direction)
    is smaller than the specified threshold
    '''
    return mockValue

def executeCameraCommand(command):
    global globalVerticalNavigationTime
    global globalIsNavigatingVertically
    global globalLastCameraComand
    global globalVerticalNavigationTs
    
    validCommands = {'stop', 'forward', 'backward', 'right', 'left', 'finish'}
    
    if command in validCommands:
        if command == 'finish':
            # Return to navigation line
            if globalVerticalNavigationTime>0:
                # Move bacward
                navigationCommand('backward', 0.1, timeOut=globalVerticalNavigationTime)
            if globalVerticalNavigationTime<0:
                # Move bacward
                navigationCommand('forward', 0.1, timeOut=-(globalVerticalNavigationTime))
            globalVerticalNavigationTime = 0
        else:
            if command in {'forward', 'backward'}:
                globalIsNavigatingVertically = True
                globalVerticalNavigationTs = time.time()
            if command == 'stop':
                if globalIsNavigatingVertically:
                    globalIsNavigatingVertically = False
                    if globalLastCameraComand == 'forward':
                        globalVerticalNavigationTime += (time.time() - globalVerticalNavigationTs)
                    if globalLastCameraComand == 'bacward':
                        globalVerticalNavigationTime -= (time.time() - globalVerticalNavigationTs)
            navigationCommand(command, 0.1)
    else:
        print('\nunkown commad')
        
    globalLastCameraComand=command
    return
    

def start(cyclesInfo):
    mainNavigationDirection = 90
    orientationDirection = 180
    
    for cycleInfo in cyclesInfo:
        # set variables used during the cycle
        endOfCycle = false
        
        globalIsNavigatingVertically=False
        globalVerticalNavigationTs=0
        globalVerticalNavigationTime=0
        globalLastCameraComand=None
        
        #Start navigation
        navigationCommand("left", 0.5)

        #Start camera
        sendCameraCommand("start")
        
        while (not endOfCycle):
            # Check if incoming bytes are waiting to be read from the serial input buffer.
            # NB: for PySerial v3.0 or later, use property `in_waiting` instead of
            # function `inWaiting()` below!
            if (ser.inWaiting() > 0):
                # read the bytes and convert from binary array to ASCII
                cameraCommand = ser.read(ser.inWaiting()).decode('ascii') 
                print(f"\nCamera command received: {cameraCommand}") 
                
                # Executing camera command
                executeCameraCommand(cameraCommand)
                
            else:
                print('\nNo camera command received')
                
            # Check end of cycle
            lidarData = getLidarData()
            endOfCycle = isEndOfCycle(lidarData, mainNavigationDirection, cycleInfo['endOfCycleDistance'], mockValue=False)
                
        # Stop arm/camera system
        executeCameraCommand('stop')
        
        # Horizontal Navigation
        navigateUsingWall(cycleInfo['finalPoint'][0]['lidarDirection'], cycleInfo['finalPoint'][0]['targetDistance'])
        
        # Vertical Navigation
        navigateUsingWall(cycleInfo['finalPoint'][1]['lidarDirection'], cycleInfo['finalPoint'][1]['targetDistance'])
        
        # Rotate
        navigationCommand('rotate_right', 0.1, timeOut=2)
        
        # Setting final orientation
        setOrientationUsingWall(orientationDirection)